In [0]:
# All Includes
from sklearn.metrics import classification_report
from keras.models import Sequential, load_model, clone_model
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import models, layers, optimizers, utils
import PIL, PIL.ImageOps, PIL.ImageEnhance, PIL.ImageDraw, PIL.Image
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import time
import random

**=== Child Model ===**

In [0]:


def create_model(trainX, n_classes):
    # https://stats.stackexchange.com/questions/272607/cifar-10-cant-get-above-60-accuracy-keras-with-tensorflow-backend
    model = Sequential()
    model.add(Convolution2D(input_shape=trainX[0,:,:,:].shape, filters=96, kernel_size=(3,3)))
    model.add(Activation('relu'))
    model.add(Convolution2D(filters=96, kernel_size=(3,3), strides=2))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(filters=192, kernel_size=(3,3)))
    model.add(Activation('relu'))
    model.add(Convolution2D(filters=192, kernel_size=(3,3), strides=2))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(n_classes, activation="softmax"))
    optimizer = optimizers.Adadelta(lr=0.05, rho=0.95, epsilon=None, decay=0.0)
    model.compile(optimizer, 'categorical_crossentropy', ['accuracy'])
    return model

  

  
def model_cond_accuracy(model, X, y):
    y_prob = model.predict(X)
    y_classes = y_prob.argmax(axis=-1).tolist()
    y_test = y.argmax(axis=-1).tolist()
    total = [0] * 10
    counts = [0] * 10
    for i in range(len(y_classes)):
      if y_classes[i] == y_test[i]:
        total[y_test[i]] += 1
      counts[y_test[i]] += 1
    acc = [0.0] * 10
    for i in range(10):
      if 0 != counts[i]:
        acc[i] = total[i] / counts[i]
    return acc

def model_fit(model, gen, val_data, nbatches, epochs):
    history = model.fit_generator(
      gen, nbatches, epochs, verbose=1, use_multiprocessing=True, validation_data =  val_data)
    return history

def model_evaluate(model, X, y):
  return model.evaluate(X, y, verbose=0)[1]

**=== Transforms ===**

In [0]:
# Code below adapted from augmentation_transforms.py
# Modified to support transforms at the image class level
# Original copywright below:

# Copyright 2018 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================



IMAGE_SIZE = 32
MEANS = [0.49139968, 0.48215841, 0.44653091]
STDS = [0.24703223, 0.24348513, 0.26158784]
PARAMETER_MAX = 10  # What is the max 'level' a transform could be predicted

def pil_wrap(img):
  """Convert the `img` numpy tensor to a PIL Image."""
  return PIL.Image.fromarray(
      np.uint8((img * STDS + MEANS) * 255.0)).convert('RGBA')


def pil_unwrap(pil_img):
  """Converts the PIL img to a numpy array."""
  pic_array = (np.array(pil_img.getdata()).reshape((32, 32, 4)) / 255.0)
  i1, i2 = np.where(pic_array[:, :, 3] == 0)
  pic_array = (pic_array[:, :, :3] - MEANS) / STDS
  pic_array[i1, i2] = [0, 0, 0]
  return pic_array

class Operation:
    def __init__(self, t, p = 0.5):

        self.prob = p
        self.magnitude = t[1]
        self.transformation = t[0]

    def __call__(self, X, Y):
        _X = []
        #self.magnitude = random.randint(0,9)
        for x,y in zip(X,Y):
            if np.random.rand() < self.prob:
                x = pil_wrap(x)
                x = self.transformation[np.argmax(y)](x, self.magnitude)
                x = pil_unwrap(x)
            _X.append(np.array(x))
        return np.array(_X)
    

class Transform:
    def __init__(self, *operations):
        self.operations = operations

    def __call__(self, X, Y):
        for op in self.operations:
            X = op(X, Y)
        return X


def autoaugment(transforms, X, y, batch_size):
    while True:
        ix = np.arange(len(X))
        np.random.shuffle(ix)
        for i in range(len(X) // batch_size):
            _ix = ix[i*batch_size:(i+1)*batch_size]
            _X = X[_ix]
            _y = y[_ix]
            if 0 != len(transforms):
              transform = np.random.choice(transforms)
              _X = transform(_X, _y)
            yield _X, _y

# modified from https://github.com/rpmcruz/autoaugment/blob/master/transformations.py
def create_cutout_mask(img_height, img_width, num_channels, size):
  """Creates a zero mask used for cutout of shape `img_height` x `img_width`.

  Args:
    img_height: Height of image cutout mask will be applied to.
    img_width: Width of image cutout mask will be applied to.
    num_channels: Number of channels in the image.
    size: Size of the zeros mask.

  Returns:
    A mask of shape `img_height` x `img_width` with all ones except for a
    square of zeros of shape `size` x `size`. This mask is meant to be
    elementwise multiplied with the original image. Additionally returns
    the `upper_coord` and `lower_coord` which specify where the cutout mask
    will be applied.
  """
  assert img_height == img_width

  # Sample center where cutout mask will be applied
  height_loc = np.random.randint(low=0, high=img_height)
  width_loc = np.random.randint(low=0, high=img_width)

  # Determine upper right and lower left corners of patch
  upper_coord = (max(0, height_loc - size // 2), max(0, width_loc - size // 2))
  lower_coord = (min(img_height, height_loc + size // 2),
                 min(img_width, width_loc + size // 2))
  mask_height = lower_coord[0] - upper_coord[0]
  mask_width = lower_coord[1] - upper_coord[1]
  assert mask_height > 0
  assert mask_width > 0

  mask = np.ones((img_height, img_width, num_channels))
  zeros = np.zeros((mask_height, mask_width, num_channels))
  mask[upper_coord[0]:lower_coord[0], upper_coord[1]:lower_coord[1], :] = (
      zeros)
  return mask, upper_coord, lower_coord

def cutout_numpy(img, size=16):
  """Apply cutout with mask of shape `size` x `size` to `img`.

  The cutout operation is from the paper https://arxiv.org/abs/1708.04552.
  This operation applies a `size`x`size` mask of zeros to a random location
  within `img`.

  Args:
    img: Numpy image that cutout will be applied to.
    size: Height/width of the cutout mask that will be

  Returns:
    A numpy tensor that is the result of applying the cutout mask to `img`.
  """
  img_height, img_width, num_channels = (img.shape[0], img.shape[1],
                                         img.shape[2])
  assert len(img.shape) == 3
  mask, _, _ = create_cutout_mask(img_height, img_width, num_channels, size)
  return img * mask

def float_parameter(level, maxval):
  """Helper function to scale `val` between 0 and maxval .

  Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled
      to level/PARAMETER_MAX.

  Returns:
    A float that results from scaling `maxval` according to `level`.
  """
  return float(level) * maxval / PARAMETER_MAX


def int_parameter(level, maxval):
  """Helper function to scale `val` between 0 and maxval .

  Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled
      to level/PARAMETER_MAX.

  Returns:
    An int that results from scaling `maxval` according to `level`.
  """
  return int(level * maxval / PARAMETER_MAX)

def _cutout_pil_impl(pil_img, level):
  """Apply cutout to pil_img at the specified level."""
  size = int_parameter(level, 20)
  if size <= 0:
    return pil_img
  img_height, img_width, num_channels = (32, 32, 3)
  _, upper_coord, lower_coord = (
      create_cutout_mask(img_height, img_width, num_channels, size))
  pixels = pil_img.load()  # create the pixel map
  for i in range(upper_coord[0], lower_coord[0]):  # for every col:
    for j in range(upper_coord[1], lower_coord[1]):  # For every row
      pixels[i, j] = (125, 122, 113, 0)  # set the colour accordingly
  return pil_img

def _enhancer_impl(enhancer):
  """Sets level to be between 0.1 and 1.8 for ImageEnhance transforms of PIL."""
  def impl(pil_img, level):
    v = float_parameter(level, 1.8) + .1  # going to 0 just destroys it
    return enhancer(pil_img).enhance(v)
  return impl

# =============================================================================

def ShearX(img, v):  # [-0.3, 0.3]
    if random.random() > 0.5:
      v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, v, 0, 0, 1, 0))
  
def ShearY(img, v):  # [-0.3, 0.3]
    if random.random() > 0.5:
      v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, v, 1, 0))  
  
def TranslateX(img, v):  # [-150, 150] => percentage: [-0.45, 0.45]
    if random.random() > 0.5:
      v = -v
    return img.transform((32, 32), PIL.Image.AFFINE, (1, 0, v, 0, 1, 0))

def TranslateY(img, v):  # [-150, 150] => percentage: [-0.45, 0.45]
    if random.random() > 0.5:
      v = -v
    return img.transform((32, 32), PIL.Image.AFFINE, (1, 0, 0, 0, 1, v))

def Rotate(img, v):  # [-30, 30]
    if random.random() > 0.5:
      v = -v
    return img.rotate(v)

def AutoContrast(img, _):
    return PIL.ImageOps.autocontrast(img.convert('RGB')).convert('RGBA')

def Invert(img, _):
    return PIL.ImageOps.invert(img.convert('RGB')).convert('RGBA')

def Equalize(img, _):
    return PIL.ImageOps.equalize(img.convert('RGB')).convert('RGBA')

def Flip_LR(img, _):  # not from the paper
    return img.transpose(PIL.Image.FLIP_LEFT_RIGHT)

def Flip_UD(img, _):
    return img.transpose(PIL.Image.FLIP_TOP_BOTTOM)
  
def Solarize(img, v):  # [0, 256]
    v = int_parameter(v, 256)
    return PIL.ImageOps.solarize(img.convert('RGB'), 256 - v).convert('RGBA')

def Posterize(img, v):  # [4, 8]
    v = int_parameter(v, 4)
    return PIL.ImageOps.posterize(img.convert('RGB'), 4 - v).convert('RGBA')

def Contrast(img, v):  # [0.1,1.9]
    return _enhancer_impl(PIL.ImageEnhance.Contrast)(img, v)

def Blur(img, v):
    return img.filter(PIL.ImageFilter.BLUR)
  
def Color(img, v):  # [0.1,1.9]
    return _enhancer_impl(PIL.ImageEnhance.Color)(img, v)

def Smooth(img, v):
    return img.filter(PIL.ImageFilter.SMOOTH)
  
def Brightness(img, v):  # [0.1,1.9]
    return _enhancer_impl(PIL.ImageEnhance.Brightness)(img, v)

def Sharpness(img, v):  # [0.1,1.9]
    return _enhancer_impl(PIL.ImageEnhance.Sharpness)(img, v)

def Cutout(img, v):  # [0, 60] => percentage: [0, 0.2]
    return _cutout_pil_impl(img, v)

def Crop(img, v, interpolation=PIL.Image.BILINEAR):
    cropped = img.crop((v, v, IMAGE_SIZE - v, IMAGE_SIZE - v))
    resized = cropped.resize((IMAGE_SIZE, IMAGE_SIZE), interpolation)
    return resized

def Identity(img, v):
  return img

  
opmap = {
    'Flip_LR' : Flip_LR,
    'Flip_UD' : Flip_UD,
    'AutoContrast' : AutoContrast,
    'Equalize' : Equalize,
    'Invert' : Invert,
    'Rotate' : Rotate,
    'Poserize' : Posterize,
    'Crop' : Crop,
    'Solarize' : Solarize,
    'Color' : Color,
    'Contrast' : Contrast,
    'Brightness' : Brightness,
    'Sharpness' : Sharpness,
    'ShearX' : ShearX,
    'ShearY' : ShearY,
    'TranslateX' : TranslateX,
    'TranslateY' : TranslateY,
    'Cutout' : Cutout,
    'Blur' : Blur,
    'Smooth' : Smooth
    
}

**=== Get Ready... ===**

In [21]:
# Load CIFAR-10
from keras.datasets import cifar10
(X, y), (X_test, y_test) = cifar10.load_data()



# Create the Reduced CIFAR-10 dataset
#ix = np.random.choice(len(X), 4000, False)
#x_reduced_train = X[ix]
#y_reduced_train = y[ix]

# Shuffle the training data
shuffling = np.random.permutation(X.shape[0])   
X = X[shuffling, :]
y = y[shuffling]

# Split Training --> Training + Validation
nTrain = int(0.9 * X.shape[0])
X_train = X[0:nTrain, :, :, :]
y_train = y[:nTrain]

X_validation = X[nTrain:, :, :, :]
y_validation = y[nTrain:]

X_reduced = X[:4000, :, :, :]
y_reduced = y[:4000]

print(X_train.shape)
print(X_validation.shape)
print(X_reduced.shape)

y_train = utils.to_categorical(y_train)
y_validation = utils.to_categorical(y_validation)
y_test = utils.to_categorical(y_test)
y_reduced = utils.to_categorical(y_reduced)

categories = ['airplane', 'auto', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

(45000, 32, 32, 3)
(5000, 32, 32, 3)
(4000, 32, 32, 3)


In [0]:
batch_size = 250

def train_model(X, model, aug, epochs):

  tic = time.clock()
  model_fit(model, aug, (X_validation, y_validation), len(X) // batch_size, epochs)
  toc = time.clock()
  
  accuracy = model_evaluate(model, X_test, y_test)
  
  print('Test accuracy: %.3f (elaspsed time: %ds)' % (accuracy, (toc-tic)))
  acc = model_cond_accuracy(model, X_test, y_test)
  
  print("Accuracy")
  for cat, a in zip(categories, acc):
    print(cat, a)

**Baseline Training Without Data Augmentation**

In [29]:
ops = [Operation(([Identity] * 10, 0), 0)]
transform1 = Transform(*ops)

transforms = [transform1]
aug = autoaugment(transforms, X_train, y_train, batch_size) 

model0 = create_model(X_train, 10)

history0 = train_model(model0, aug, 50)

Epoch 1/50
180/180 [==============================] - 19s 107ms/step - loss: 2.3150 - acc: 0.2157 - val_loss: 1.9072 - val_acc: 0.3130
Epoch 2/50
180/180 [==============================] - 18s 97ms/step - loss: 1.9128 - acc: 0.3246 - val_loss: 1.6609 - val_acc: 0.4032
Epoch 3/50
180/180 [==============================] - 18s 97ms/step - loss: 1.7215 - acc: 0.3860 - val_loss: 1.6090 - val_acc: 0.4246
Epoch 4/50
180/180 [==============================] - 18s 97ms/step - loss: 1.5929 - acc: 0.4275 - val_loss: 1.4410 - val_acc: 0.4826
Epoch 5/50
180/180 [==============================] - 17s 97ms/step - loss: 1.5025 - acc: 0.4597 - val_loss: 1.3629 - val_acc: 0.5158
Epoch 6/50
180/180 [==============================] - 17s 97ms/step - loss: 1.4435 - acc: 0.4834 - val_loss: 1.3087 - val_acc: 0.5280
Epoch 7/50
180/180 [==============================] - 17s 96ms/step - loss: 1.3849 - acc: 0.5032 - val_loss: 1.2902 - val_acc: 0.5464
Epoch 8/50
180/180 [==============================] - 17s 96m

**Train for each individual image transform**
Do transforms have significantly different effects for each image class?

In [18]:
# Try one to see how many epochs to try... 100 seems to work well; need lots of extra training!
op = opmap['Flip_LR']
print('=========================================================')
print('=========================================================')
print('=== ', 'Flip_LR', ' ===')

# 50% prob of transform
ops = [Operation(([op] * 10, 5), 0.5)]
transform = [Transform(*ops)]

aug = autoaugment(transform, X_train, y_train, batch_size) 

model = create_model(X_train, 10)

history = train_model(model, aug, 100)
  

===  Flip_LR  ===
Epoch 1/100


TypeError: ignored

In [13]:
for name, op in opmap.items():
  print('=========================================================')
  print('=========================================================')
  print('=== ', name, ' ===')

  # 50% prob of transform
  ops = [Operation(([op] * 10, 5), 0.5)]
  transform = [Transform(*ops)]

  aug = autoaugment(transform, X_train, y_train, batch_size) 
  
  model = create_model(X_train, 10)

  history = train_model(model, aug, 100)

===  Flip_LR  ===
Epoch 1/100
 74/180 [===========>..................] - ETA: 20s - loss: 2.5491 - acc: 0.1252

Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 626, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "<ipython-input-4-d4f43fd572ab>", line 62, in autoaugment
    _X = transform(_X, _y)
  File "<ipython-input-4-d4f43fd572ab>", line 48, in __call__
    X = op(X, Y)
  File "<ipython-input-4-d4f43fd572ab>", line 37, in __call__
    x = pil_unwrap(x)
KeyboardInterrupt
  File "<ipython-input-4-d4f43fd572ab>", line 19, in pil_unwrap
    pic_array = (pic_array[:, :, :3] - MEANS) / STDS


Epoch 1/100


KeyboardInterrupt: ignored

**AutoAugment Policy**

In [7]:
# Utility function to create policy, tform, below
def mk_op2(op1, p1, v1, op2, p2, v2):
  ops = [Operation(([op1] * 10, v1), p1), Operation(([op2] * 10, v2), p2)]
  return Transform(*ops)

# Duplicate the AutoAugment CIFAR-10 policy selected by concatenations of AutoAugment
tform = [
    #0_0
    mk_op2(Invert, 0.1, 7, Contrast, 0.2, 6),
    mk_op2(Rotate, 0.7, 2, TranslateX, 0.3, 9),
    mk_op2(Sharpness, 0.8, 1, Sharpness, 0.9, 3),
    mk_op2(ShearY, 0.5, 8, TranslateY, 0.7, 9),
    mk_op2(AutoContrast, 0.5, 8, Equalize, 0.9, 2),
    
    #1_0
    mk_op2(ShearY, 0.2, 7, Posterize, 0.3, 7),
    mk_op2(Color, 0.4, 3, Brightness, 0.6, 7),
    mk_op2(Sharpness, 0.3, 9, Brightness, 0.7, 9),
    mk_op2(Equalize, 0.6, 5, Equalize, 0.5, 1),
    mk_op2(Contrast, 0.6, 7, Sharpness, 0.6, 5),
    
    #2_0
    mk_op2(Color, 0.7, 7, TranslateX, 0.5, 8),
    mk_op2(Equalize, 0.3, 7, AutoContrast, 0.4, 8),
    mk_op2(TranslateY, 0.4, 3, Sharpness, 0.2, 6),
    mk_op2(Brightness, 0.9, 6, Color, 0.2, 8),
    mk_op2(Solarize, 0.5, 2, Invert, 0.0, 3),
    
    #3_0
    mk_op2(Equalize,0.2,0, AutoContrast,0.6,0),
    mk_op2(Equalize,0.2,8, Equalize,0.6,4),
    mk_op2(Color,0.9,9, Equalize,0.6,6),
    mk_op2(AutoContrast,0.8,4, Solarize,0.2,8),
    mk_op2(Brightness,0.1,3, Color,0.7,0),
    
    #4_0
    mk_op2(Solarize,0.4,5, AutoContrast,0.9,3),
    mk_op2(TranslateY,0.9,9, TranslateY,0.7,9),
    mk_op2(AutoContrast,0.9,2, Solarize,0.8,3),
    mk_op2(Equalize,0.8,8, Invert,0.1,3),
    mk_op2(TranslateY,0.7,9, AutoContrast,0.9,1)
]


aug = autoaugment(tform, X_train, y_train, batch_size) 

model = create_model(X_train, 10)

train_model(model, aug, 100)

Epoch 1/100
180/180 [==============================] - 70s 389ms/step - loss: 2.4581 - acc: 0.1135 - val_loss: 2.0783 - val_acc: 0.2264
Epoch 2/100
180/180 [==============================] - 62s 347ms/step - loss: 2.3042 - acc: 0.1370 - val_loss: 2.0000 - val_acc: 0.2642
Epoch 3/100
180/180 [==============================] - 68s 375ms/step - loss: 2.2584 - acc: 0.1450 - val_loss: 1.9712 - val_acc: 0.2750
Epoch 4/100
180/180 [==============================] - 65s 362ms/step - loss: 2.2337 - acc: 0.1486 - val_loss: 1.9056 - val_acc: 0.3208
Epoch 5/100
180/180 [==============================] - 67s 372ms/step - loss: 2.2156 - acc: 0.1559 - val_loss: 1.8762 - val_acc: 0.3032
Epoch 6/100
180/180 [==============================] - 68s 378ms/step - loss: 2.2049 - acc: 0.1587 - val_loss: 1.8288 - val_acc: 0.3308
Epoch 7/100
180/180 [==============================] - 68s 376ms/step - loss: 2.1869 - acc: 0.1678 - val_loss: 1.7608 - val_acc: 0.3702
Epoch 8/100
180/180 [===========================

In [9]:
# Train some more!
train_model(model, aug, 50)

Epoch 1/50
180/180 [==============================] - 72s 401ms/step - loss: 1.8773 - acc: 0.2979 - val_loss: 0.9740 - val_acc: 0.6722
Epoch 2/50
180/180 [==============================] - 67s 371ms/step - loss: 1.7850 - acc: 0.3317 - val_loss: 0.8716 - val_acc: 0.7034
Epoch 3/50
180/180 [==============================] - 71s 396ms/step - loss: 1.8211 - acc: 0.3178 - val_loss: 0.8782 - val_acc: 0.6956
Epoch 4/50
180/180 [==============================] - 69s 383ms/step - loss: 1.8184 - acc: 0.3182 - val_loss: 0.9371 - val_acc: 0.6832
Epoch 5/50
180/180 [==============================] - 70s 387ms/step - loss: 1.8349 - acc: 0.3130 - val_loss: 0.8924 - val_acc: 0.6952
Epoch 6/50
180/180 [==============================] - 67s 375ms/step - loss: 1.8439 - acc: 0.3077 - val_loss: 0.8524 - val_acc: 0.7088
Epoch 7/50
180/180 [==============================] - 68s 379ms/step - loss: 1.8352 - acc: 0.3104 - val_loss: 0.8808 - val_acc: 0.7002
Epoch 8/50
180/180 [==============================] - 6

In [10]:
# Train some more!
train_model(model, aug, 50)

Epoch 1/50
180/180 [==============================] - 74s 412ms/step - loss: 1.8126 - acc: 0.3222 - val_loss: 0.7744 - val_acc: 0.7360
Epoch 2/50
180/180 [==============================] - 68s 377ms/step - loss: 1.7033 - acc: 0.3661 - val_loss: 0.7893 - val_acc: 0.7330
Epoch 3/50
180/180 [==============================] - 70s 388ms/step - loss: 1.7516 - acc: 0.3462 - val_loss: 0.7894 - val_acc: 0.7378
Epoch 4/50
180/180 [==============================] - 65s 364ms/step - loss: 1.7548 - acc: 0.3468 - val_loss: 0.8557 - val_acc: 0.7160
Epoch 5/50
180/180 [==============================] - 67s 370ms/step - loss: 1.7666 - acc: 0.3411 - val_loss: 0.7885 - val_acc: 0.7324
Epoch 6/50
180/180 [==============================] - 67s 374ms/step - loss: 1.7744 - acc: 0.3386 - val_loss: 0.7702 - val_acc: 0.7434
Epoch 7/50
180/180 [==============================] - 67s 374ms/step - loss: 1.7702 - acc: 0.3412 - val_loss: 0.7891 - val_acc: 0.7376
Epoch 8/50
180/180 [==============================] - 6

**Common Baseline**
Above results are like comparing Red Deliscious Apples to Granny Smith Apples
Try a common baseline here

In [24]:
#No=op policy
ops = [Operation(([Identity] * 10, 0), 0)]
transform1 = Transform(*ops)

transforms = [transform1]
aug = autoaugment(transforms, X_reduced, y_reduced, batch_size) 

base = create_model(X_reduced, 10)

train_model(X_reduced, base, aug, 10)

Epoch 1/10
16/16 [==============================] - 4s 273ms/step - loss: 2.9350 - acc: 0.1115 - val_loss: 2.1769 - val_acc: 0.1888
Epoch 2/10
16/16 [==============================] - 2s 121ms/step - loss: 2.6319 - acc: 0.1560 - val_loss: 2.0965 - val_acc: 0.2206
Epoch 3/10
16/16 [==============================] - 2s 121ms/step - loss: 2.4512 - acc: 0.1790 - val_loss: 2.1650 - val_acc: 0.2178
Epoch 4/10
16/16 [==============================] - 2s 122ms/step - loss: 2.3121 - acc: 0.2165 - val_loss: 2.2085 - val_acc: 0.2264
Epoch 5/10
16/16 [==============================] - 2s 122ms/step - loss: 2.2372 - acc: 0.2342 - val_loss: 2.2289 - val_acc: 0.2324
Epoch 6/10
16/16 [==============================] - 2s 123ms/step - loss: 2.2136 - acc: 0.2455 - val_loss: 2.1214 - val_acc: 0.2612
Epoch 7/10
16/16 [==============================] - 2s 122ms/step - loss: 2.1438 - acc: 0.2575 - val_loss: 1.9786 - val_acc: 0.3082
Epoch 8/10
16/16 [==============================] - 2s 121ms/step - loss: 2.

Finish training the baseline...

In [25]:
base_final = create_model(X_reduced, 10)
base_final.set_weights(base.get_weights())

train_model(X_reduced, base_final, aug, 40)

Epoch 1/40
16/16 [==============================] - 4s 278ms/step - loss: 1.9321 - acc: 0.3185 - val_loss: 1.8264 - val_acc: 0.3584
Epoch 2/40
16/16 [==============================] - 2s 124ms/step - loss: 1.8903 - acc: 0.3427 - val_loss: 1.7754 - val_acc: 0.3736
Epoch 3/40
16/16 [==============================] - 2s 123ms/step - loss: 1.8385 - acc: 0.3405 - val_loss: 1.7752 - val_acc: 0.3772
Epoch 4/40
16/16 [==============================] - 2s 123ms/step - loss: 1.8105 - acc: 0.3668 - val_loss: 1.7363 - val_acc: 0.3924
Epoch 5/40
16/16 [==============================] - 2s 123ms/step - loss: 1.7723 - acc: 0.3752 - val_loss: 1.7718 - val_acc: 0.3756
Epoch 6/40
16/16 [==============================] - 2s 123ms/step - loss: 1.7194 - acc: 0.3853 - val_loss: 1.7464 - val_acc: 0.3788
Epoch 7/40
16/16 [==============================] - 2s 123ms/step - loss: 1.6949 - acc: 0.3913 - val_loss: 1.6721 - val_acc: 0.4114
Epoch 8/40
16/16 [==============================] - 2s 123ms/step - loss: 1.

In [30]:
train_model(X_reduced, base_final, aug, 50)

Epoch 1/50
16/16 [==============================] - 2s 124ms/step - loss: 0.9611 - acc: 0.6648 - val_loss: 1.5428 - val_acc: 0.4622
Epoch 2/50
16/16 [==============================] - 2s 124ms/step - loss: 0.9377 - acc: 0.6707 - val_loss: 1.5092 - val_acc: 0.4736
Epoch 3/50
16/16 [==============================] - 2s 125ms/step - loss: 0.9212 - acc: 0.6835 - val_loss: 1.5333 - val_acc: 0.4734
Epoch 4/50
16/16 [==============================] - 2s 123ms/step - loss: 0.8976 - acc: 0.6885 - val_loss: 1.5204 - val_acc: 0.4714
Epoch 5/50
16/16 [==============================] - 2s 124ms/step - loss: 0.8918 - acc: 0.6962 - val_loss: 1.4804 - val_acc: 0.4804
Epoch 6/50
16/16 [==============================] - 2s 124ms/step - loss: 0.8673 - acc: 0.6942 - val_loss: 1.5596 - val_acc: 0.4594
Epoch 7/50
16/16 [==============================] - 2s 124ms/step - loss: 0.8730 - acc: 0.6947 - val_loss: 1.4626 - val_acc: 0.4878
Epoch 8/50
16/16 [==============================] - 2s 124ms/step - loss: 0.

In [31]:
train_model(X_reduced, base_final, aug, 50)

Epoch 1/50
16/16 [==============================] - 2s 135ms/step - loss: 0.3953 - acc: 0.8845 - val_loss: 1.4801 - val_acc: 0.5188
Epoch 2/50
16/16 [==============================] - 2s 123ms/step - loss: 0.3862 - acc: 0.8840 - val_loss: 1.4740 - val_acc: 0.5224
Epoch 3/50
16/16 [==============================] - 2s 125ms/step - loss: 0.3836 - acc: 0.8892 - val_loss: 1.5035 - val_acc: 0.5108
Epoch 4/50
16/16 [==============================] - 2s 124ms/step - loss: 0.3878 - acc: 0.8848 - val_loss: 1.4855 - val_acc: 0.5184
Epoch 5/50
16/16 [==============================] - 2s 125ms/step - loss: 0.3750 - acc: 0.8900 - val_loss: 1.4867 - val_acc: 0.5184
Epoch 6/50
16/16 [==============================] - 2s 126ms/step - loss: 0.3532 - acc: 0.9045 - val_loss: 1.4812 - val_acc: 0.5276
Epoch 7/50
16/16 [==============================] - 2s 125ms/step - loss: 0.3517 - acc: 0.8995 - val_loss: 1.4698 - val_acc: 0.5214
Epoch 8/50
16/16 [==============================] - 2s 125ms/step - loss: 0.

Now try train further with autoaugment...

Note: train data is size 45,000 and minibatch size is 250 so that each epoch is exposed to 180 different sub-policies
With 95 nearly unique sub-policies, each epoch is exposed to most or all sub-policies.

In [39]:
CIFAR10_Policy = [
    #0_0
    mk_op2(Invert, 0.1, 7, Contrast, 0.2, 6),
    mk_op2(Rotate, 0.7, 2, TranslateX, 0.3, 9),
    mk_op2(Sharpness, 0.8, 1, Sharpness, 0.9, 3),
    mk_op2(ShearY, 0.5, 8, TranslateY, 0.7, 9),
    mk_op2(AutoContrast, 0.5, 8, Equalize, 0.9, 2),
    
    #1_0
    mk_op2(ShearY, 0.2, 7, Posterize, 0.3, 7),
    mk_op2(Color, 0.4, 3, Brightness, 0.6, 7),
    mk_op2(Sharpness, 0.3, 9, Brightness, 0.7, 9),
    mk_op2(Equalize, 0.6, 5, Equalize, 0.5, 1),
    mk_op2(Contrast, 0.6, 7, Sharpness, 0.6, 5),
    
    #2_0
    mk_op2(Color, 0.7, 7, TranslateX, 0.5, 8),
    mk_op2(Equalize, 0.3, 7, AutoContrast, 0.4, 8),
    mk_op2(TranslateY, 0.4, 3, Sharpness, 0.2, 6),
    mk_op2(Brightness, 0.9, 6, Color, 0.2, 8),
    mk_op2(Solarize, 0.5, 2, Invert, 0.0, 3),
    
    #3_0
    mk_op2(Equalize,0.2,0, AutoContrast,0.6,0),
    mk_op2(Equalize,0.2,8, Equalize,0.6,4),
    mk_op2(Color,0.9,9, Equalize,0.6,6),
    mk_op2(AutoContrast,0.8,4, Solarize,0.2,8),
    mk_op2(Brightness,0.1,3, Color,0.7,0),
    
    #4_0
    mk_op2(Solarize,0.4,5, AutoContrast,0.9,3),
    mk_op2(TranslateY,0.9,9, TranslateY,0.7,9),
    mk_op2(AutoContrast,0.9,2, Solarize,0.8,3),
    mk_op2(Equalize,0.8,8, Invert,0.1,3),
    mk_op2(TranslateY,0.7,9, AutoContrast,0.9,1)
]


aug_autoaug = autoaugment(CIFAR10_Policy, X_reduced, y_reduced, batch_size) 

# create new model and copy base weights over
model_autoaug = create_model(X_reduced, 10)
model_autoaug.set_weights(base.get_weights())

train_model(X_reduced, model_autoaug, aug_autoaug, 100)

Epoch 1/100
16/16 [==============================] - 9s 573ms/step - loss: 2.4502 - acc: 0.1425 - val_loss: 2.0993 - val_acc: 0.3296
Epoch 2/100
16/16 [==============================] - 7s 440ms/step - loss: 2.3739 - acc: 0.1375 - val_loss: 2.1054 - val_acc: 0.3536
Epoch 3/100
16/16 [==============================] - 7s 435ms/step - loss: 2.3654 - acc: 0.1405 - val_loss: 2.1375 - val_acc: 0.3320
Epoch 4/100
16/16 [==============================] - 7s 423ms/step - loss: 2.3624 - acc: 0.1368 - val_loss: 2.0098 - val_acc: 0.3408
Epoch 5/100
16/16 [==============================] - 7s 420ms/step - loss: 2.3437 - acc: 0.1465 - val_loss: 2.0485 - val_acc: 0.3296
Epoch 6/100
16/16 [==============================] - 6s 385ms/step - loss: 2.3166 - acc: 0.1520 - val_loss: 1.9614 - val_acc: 0.3428
Epoch 7/100
16/16 [==============================] - 6s 373ms/step - loss: 2.3267 - acc: 0.1455 - val_loss: 1.8910 - val_acc: 0.3510
Epoch 8/100
16/16 [==============================] - 7s 417ms/step - 

In [0]:
# train more...
train_model(X_reduced, model_autoaug, aug_autoaug, 200)

Epoch 1/200
16/16 [==============================] - 7s 438ms/step - loss: 2.0764 - acc: 0.2405 - val_loss: 1.6512 - val_acc: 0.4208
Epoch 2/200
16/16 [==============================] - 7s 437ms/step - loss: 2.1110 - acc: 0.2190 - val_loss: 1.6287 - val_acc: 0.4292
Epoch 3/200
16/16 [==============================] - 7s 447ms/step - loss: 2.1144 - acc: 0.2272 - val_loss: 1.6380 - val_acc: 0.4114
Epoch 4/200
16/16 [==============================] - 7s 412ms/step - loss: 2.0581 - acc: 0.2455 - val_loss: 1.6035 - val_acc: 0.4282
Epoch 5/200
16/16 [==============================] - 7s 414ms/step - loss: 2.0638 - acc: 0.2470 - val_loss: 1.6104 - val_acc: 0.4278
Epoch 6/200
16/16 [==============================] - 6s 382ms/step - loss: 2.0513 - acc: 0.2515 - val_loss: 1.6403 - val_acc: 0.4094
Epoch 7/200
16/16 [==============================] - 6s 370ms/step - loss: 2.0540 - acc: 0.2475 - val_loss: 1.6124 - val_acc: 0.4256
Epoch 8/200
16/16 [==============================] - 7s 415ms/step - 

In [28]:
# train more...
train_model(X_reduced, model_autoaug, aug_autoaug, 50)

Epoch 1/50
16/16 [==============================] - 7s 464ms/step - loss: 1.7893 - acc: 0.3825 - val_loss: 1.5263 - val_acc: 0.4794
Epoch 2/50
16/16 [==============================] - 6s 344ms/step - loss: 1.6762 - acc: 0.4053 - val_loss: 1.5389 - val_acc: 0.4710
Epoch 3/50
16/16 [==============================] - 7s 438ms/step - loss: 1.7925 - acc: 0.3745 - val_loss: 1.5060 - val_acc: 0.4758
Epoch 4/50
16/16 [==============================] - 6s 405ms/step - loss: 1.7172 - acc: 0.4090 - val_loss: 1.5440 - val_acc: 0.4852
Epoch 5/50
16/16 [==============================] - 5s 341ms/step - loss: 1.6094 - acc: 0.4415 - val_loss: 1.5366 - val_acc: 0.4728
Epoch 6/50
16/16 [==============================] - 6s 387ms/step - loss: 1.7220 - acc: 0.4102 - val_loss: 1.5154 - val_acc: 0.4836
Epoch 7/50
16/16 [==============================] - 6s 373ms/step - loss: 1.7162 - acc: 0.3987 - val_loss: 1.5074 - val_acc: 0.4872
Epoch 8/50
16/16 [==============================] - 5s 332ms/step - loss: 1.

In [29]:
# train more...
train_model(X_reduced, model_autoaug, aug_autoaug, 50)

Epoch 1/50
16/16 [==============================] - 7s 454ms/step - loss: 1.5838 - acc: 0.4700 - val_loss: 1.4908 - val_acc: 0.4886
Epoch 2/50
16/16 [==============================] - 6s 350ms/step - loss: 1.5429 - acc: 0.4675 - val_loss: 1.4955 - val_acc: 0.4846
Epoch 3/50
16/16 [==============================] - 7s 443ms/step - loss: 1.6076 - acc: 0.4565 - val_loss: 1.5014 - val_acc: 0.4894
Epoch 4/50
16/16 [==============================] - 7s 415ms/step - loss: 1.5436 - acc: 0.4813 - val_loss: 1.4961 - val_acc: 0.4908
Epoch 5/50
16/16 [==============================] - 6s 348ms/step - loss: 1.4081 - acc: 0.5375 - val_loss: 1.5003 - val_acc: 0.4888
Epoch 6/50
16/16 [==============================] - 6s 398ms/step - loss: 1.5508 - acc: 0.4765 - val_loss: 1.4950 - val_acc: 0.4896
Epoch 7/50
16/16 [==============================] - 6s 385ms/step - loss: 1.5252 - acc: 0.4870 - val_loss: 1.5166 - val_acc: 0.4928
Epoch 8/50
16/16 [==============================] - 5s 333ms/step - loss: 1.

In [32]:
# train more...
train_model(X_reduced, model_autoaug, aug_autoaug, 50)

Epoch 1/50
16/16 [==============================] - 7s 465ms/step - loss: 1.4180 - acc: 0.5448 - val_loss: 1.4911 - val_acc: 0.5106
Epoch 2/50
16/16 [==============================] - 6s 349ms/step - loss: 1.3939 - acc: 0.5340 - val_loss: 1.5347 - val_acc: 0.5014
Epoch 3/50
16/16 [==============================] - 7s 451ms/step - loss: 1.4507 - acc: 0.5193 - val_loss: 1.5213 - val_acc: 0.5006
Epoch 4/50
16/16 [==============================] - 7s 419ms/step - loss: 1.3364 - acc: 0.5665 - val_loss: 1.4912 - val_acc: 0.5036
Epoch 5/50
16/16 [==============================] - 6s 349ms/step - loss: 1.2359 - acc: 0.5960 - val_loss: 1.5086 - val_acc: 0.4974
Epoch 6/50
16/16 [==============================] - 6s 398ms/step - loss: 1.3578 - acc: 0.5490 - val_loss: 1.5228 - val_acc: 0.5030
Epoch 7/50
16/16 [==============================] - 6s 384ms/step - loss: 1.3552 - acc: 0.5545 - val_loss: 1.4833 - val_acc: 0.5056
Epoch 8/50
16/16 [==============================] - 5s 339ms/step - loss: 1.

Do the same for each transform in isolation...

In [33]:
# Try one to see how many epochs to try... 
op = opmap['Flip_LR']

# 50% prob of transform
ops = [Operation(([op] * 10, 5), 0.5)]
transform = [Transform(*ops)]

aug_fliplr = autoaugment(transform, X_reduced, y_reduced, batch_size) 

model_fliplr = create_model(X_reduced, 10)
model_fliplr.set_weights(base.get_weights())

history = train_model(X_reduced, model_fliplr, aug_fliplr, 300)

Epoch 1/300
16/16 [==============================] - 6s 377ms/step - loss: 2.3316 - acc: 0.1917 - val_loss: 1.8594 - val_acc: 0.3600
Epoch 2/300
16/16 [==============================] - 3s 181ms/step - loss: 2.1970 - acc: 0.1950 - val_loss: 1.8433 - val_acc: 0.3590
Epoch 3/300
16/16 [==============================] - 3s 209ms/step - loss: 2.1766 - acc: 0.2070 - val_loss: 1.7979 - val_acc: 0.3806
Epoch 4/300
16/16 [==============================] - 3s 208ms/step - loss: 2.1392 - acc: 0.2238 - val_loss: 1.8590 - val_acc: 0.3732
Epoch 5/300
16/16 [==============================] - 3s 206ms/step - loss: 2.1363 - acc: 0.2245 - val_loss: 1.8012 - val_acc: 0.3780
Epoch 6/300
16/16 [==============================] - 3s 206ms/step - loss: 2.1593 - acc: 0.2080 - val_loss: 1.7677 - val_acc: 0.3794
Epoch 7/300
16/16 [==============================] - 3s 210ms/step - loss: 2.1323 - acc: 0.2133 - val_loss: 1.7286 - val_acc: 0.3946
Epoch 8/300
16/16 [==============================] - 3s 209ms/step - 

In [34]:
# Try one to see how many epochs to try... 
op = opmap['Cutout']

# 50% prob of transform
ops = [Operation(([op] * 10, 5), 0.5)]
transform = [Transform(*ops)]

aug_cutout = autoaugment(transform, X_reduced, y_reduced, batch_size) 

model_cutout = create_model(X_reduced, 10)
model_cutout.set_weights(base.get_weights())

history = train_model(X_reduced, model_cutout, aug_cutout, 300)

Epoch 1/300
16/16 [==============================] - 6s 384ms/step - loss: 2.2986 - acc: 0.1995 - val_loss: 1.8615 - val_acc: 0.3700
Epoch 2/300
16/16 [==============================] - 3s 191ms/step - loss: 2.2051 - acc: 0.2048 - val_loss: 1.8320 - val_acc: 0.3624
Epoch 3/300
16/16 [==============================] - 3s 212ms/step - loss: 2.1961 - acc: 0.2038 - val_loss: 1.8168 - val_acc: 0.3666
Epoch 4/300
16/16 [==============================] - 3s 213ms/step - loss: 2.1724 - acc: 0.2110 - val_loss: 1.8240 - val_acc: 0.3724
Epoch 5/300
16/16 [==============================] - 3s 216ms/step - loss: 2.1660 - acc: 0.2157 - val_loss: 1.7610 - val_acc: 0.3812
Epoch 6/300
16/16 [==============================] - 3s 214ms/step - loss: 2.1325 - acc: 0.2250 - val_loss: 1.7575 - val_acc: 0.3820
Epoch 7/300
16/16 [==============================] - 3s 211ms/step - loss: 2.1109 - acc: 0.2197 - val_loss: 1.7349 - val_acc: 0.3960
Epoch 8/300
16/16 [==============================] - 3s 211ms/step - 

In [35]:
# Try one to see how many epochs to try... 
op = opmap['TranslateY']

# 50% prob of transform
ops = [Operation(([op] * 10, 5), 0.5)]
transform = [Transform(*ops)]

aug_translate_y = autoaugment(transform, X_reduced, y_reduced, batch_size) 

model_translate_y = create_model(X_reduced, 10)
model_translate_y.set_weights(base.get_weights())

history = train_model(X_reduced, model_translate_y, aug_translate_y, 300)

Epoch 1/300
16/16 [==============================] - 6s 388ms/step - loss: 2.3039 - acc: 0.1963 - val_loss: 1.8583 - val_acc: 0.3602
Epoch 2/300
16/16 [==============================] - 3s 196ms/step - loss: 2.2109 - acc: 0.1935 - val_loss: 1.8352 - val_acc: 0.3740
Epoch 3/300
16/16 [==============================] - 3s 211ms/step - loss: 2.1824 - acc: 0.2130 - val_loss: 1.8095 - val_acc: 0.3714
Epoch 4/300
16/16 [==============================] - 3s 212ms/step - loss: 2.1750 - acc: 0.2078 - val_loss: 1.8254 - val_acc: 0.3576
Epoch 5/300
16/16 [==============================] - 3s 214ms/step - loss: 2.1477 - acc: 0.2033 - val_loss: 1.7976 - val_acc: 0.3682
Epoch 6/300
16/16 [==============================] - 3s 213ms/step - loss: 2.1229 - acc: 0.2260 - val_loss: 1.7654 - val_acc: 0.3856
Epoch 7/300
16/16 [==============================] - 4s 220ms/step - loss: 2.1294 - acc: 0.2178 - val_loss: 1.7535 - val_acc: 0.3846
Epoch 8/300
16/16 [==============================] - 3s 207ms/step - 

In [36]:
# How about combining two transforms that increased accuracy (AND)?
op1 = opmap['TranslateY']
op2 = opmap['Flip_LR']
# 50% prob of transform
ops = [Operation(([op1] * 10, 5), 0.5), Operation(([op2] * 10, 5), 0.5)]
transform = [Transform(*ops)]

aug_2_tform = autoaugment(transform, X_reduced, y_reduced, batch_size) 

model_2_tform= create_model(X_reduced, 10)
model_2_tform.set_weights(base.get_weights())

history = train_model(X_reduced, model_2_tform, aug_2_tform, 300)

Epoch 1/300
16/16 [==============================] - 8s 526ms/step - loss: 2.4575 - acc: 0.1395 - val_loss: 1.9832 - val_acc: 0.3334
Epoch 2/300
16/16 [==============================] - 6s 351ms/step - loss: 2.3394 - acc: 0.1578 - val_loss: 1.9197 - val_acc: 0.3478
Epoch 3/300
16/16 [==============================] - 6s 366ms/step - loss: 2.3327 - acc: 0.1388 - val_loss: 1.9167 - val_acc: 0.3430
Epoch 4/300
16/16 [==============================] - 6s 362ms/step - loss: 2.3124 - acc: 0.1530 - val_loss: 1.8873 - val_acc: 0.3540
Epoch 5/300
16/16 [==============================] - 6s 362ms/step - loss: 2.3160 - acc: 0.1505 - val_loss: 1.9486 - val_acc: 0.3464
Epoch 6/300
16/16 [==============================] - 6s 365ms/step - loss: 2.2718 - acc: 0.1510 - val_loss: 1.8406 - val_acc: 0.3594
Epoch 7/300
16/16 [==============================] - 6s 367ms/step - loss: 2.2734 - acc: 0.1530 - val_loss: 1.9373 - val_acc: 0.3432
Epoch 8/300
16/16 [==============================] - 6s 361ms/step - 

In [37]:
# Did autoaug decrease accuracy because of saturation (too many policies for reduced dataset)? Try a non-concat policy
def mk_op2(op1, p1, v1, op2, p2, v2):
  ops = [Operation(([op1] * 10, v1), p1), Operation(([op2] * 10, v2), p2)]
  return Transform(*ops)

# Duplicate the AutoAugment CIFAR-10 policy selected by concatenations of AutoAugment
pol_autoaug0_0 = [
    #0_0
    mk_op2(Invert, 0.1, 7, Contrast, 0.2, 6),
    mk_op2(Rotate, 0.7, 2, TranslateX, 0.3, 9),
    mk_op2(Sharpness, 0.8, 1, Sharpness, 0.9, 3),
    mk_op2(ShearY, 0.5, 8, TranslateY, 0.7, 9),
    mk_op2(AutoContrast, 0.5, 8, Equalize, 0.9, 2)]

aug_autoaug0_0 = autoaugment(pol_autoaug0_0, X_reduced, y_reduced, batch_size) 

model_autoaug0_0= create_model(X_reduced, 10)
model_autoaug0_0.set_weights(base.get_weights())

history = train_model(X_reduced, model_autoaug0_0, aug_autoaug0_0, 300)

Epoch 1/300
16/16 [==============================] - 9s 575ms/step - loss: 2.4314 - acc: 0.1408 - val_loss: 2.0595 - val_acc: 0.3290
Epoch 2/300
16/16 [==============================] - 7s 410ms/step - loss: 2.3442 - acc: 0.1442 - val_loss: 1.9516 - val_acc: 0.3540
Epoch 3/300
16/16 [==============================] - 7s 414ms/step - loss: 2.3558 - acc: 0.1455 - val_loss: 1.9852 - val_acc: 0.3268
Epoch 4/300
16/16 [==============================] - 7s 447ms/step - loss: 2.3465 - acc: 0.1345 - val_loss: 2.0392 - val_acc: 0.3436
Epoch 5/300
16/16 [==============================] - 7s 427ms/step - loss: 2.3202 - acc: 0.1400 - val_loss: 2.1978 - val_acc: 0.3274
Epoch 6/300
16/16 [==============================] - 8s 486ms/step - loss: 2.3661 - acc: 0.1333 - val_loss: 2.0790 - val_acc: 0.3394
Epoch 7/300
16/16 [==============================] - 6s 397ms/step - loss: 2.3082 - acc: 0.1503 - val_loss: 2.1158 - val_acc: 0.3292
Epoch 8/300
16/16 [==============================] - 7s 415ms/step - 

In [38]:
# How about combining two transforms that increased accuracy (OR)?
op1 = opmap['TranslateY']
op2 = opmap['Flip_LR']
# 50% prob of transform
op1 = [Operation(([op1] * 10, 5), 0.5)]
op2 = [Operation(([op2] * 10, 5), 0.5)]

transform = [Transform(*op1), Transform(*op2)]

aug_2_tform_or = autoaugment(transform, X_reduced, y_reduced, batch_size) 

model_2_tform_or= create_model(X_reduced, 10)
model_2_tform_or.set_weights(base.get_weights())

history = train_model(X_reduced, model_2_tform_or, aug_2_tform_or, 300)

Epoch 1/300
16/16 [==============================] - 7s 428ms/step - loss: 2.3154 - acc: 0.1968 - val_loss: 1.8639 - val_acc: 0.3566
Epoch 2/300
16/16 [==============================] - 3s 180ms/step - loss: 2.2101 - acc: 0.1987 - val_loss: 1.8430 - val_acc: 0.3560
Epoch 3/300
16/16 [==============================] - 3s 206ms/step - loss: 2.1784 - acc: 0.2108 - val_loss: 1.8609 - val_acc: 0.3542
Epoch 4/300
16/16 [==============================] - 3s 209ms/step - loss: 2.1575 - acc: 0.2172 - val_loss: 1.7907 - val_acc: 0.3688
Epoch 5/300
16/16 [==============================] - 3s 212ms/step - loss: 2.1830 - acc: 0.2005 - val_loss: 1.8772 - val_acc: 0.3516
Epoch 6/300
16/16 [==============================] - 3s 211ms/step - loss: 2.1532 - acc: 0.2113 - val_loss: 1.7609 - val_acc: 0.3766
Epoch 7/300
16/16 [==============================] - 3s 209ms/step - loss: 2.1089 - acc: 0.2195 - val_loss: 1.7774 - val_acc: 0.3818
Epoch 8/300
16/16 [==============================] - 3s 210ms/step - 